In [ ]:
#Nate 뉴스기사 제목 스크래핑하기

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

def scrape_nate_news():
    # 섹션 정보 (섹션명: URL)
    sections = {
        "최신뉴스": "https://news.nate.com/recent?mid=n0100",
        "정치": "https://news.nate.com/recent?mid=n0200",
        "경제": "https://news.nate.com/recent?mid=n0300",
        "사회": "https://news.nate.com/recent?mid=n0400",
        "세계": "https://news.nate.com/recent?mid=n0500",
        "IT/과학": "https://news.nate.com/recent?mid=n0600"
    }
    
    for section_name, section_url in sections.items():
        print(f"\n=== {section_name} 섹션 ===")
        
        try:
            # 페이지 요청
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(section_url, headers=headers)
            response.raise_for_status()
            
            # HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 기사 목록 추출
            articles = soup.select('.mduSubjectList .mlt01')
            
            for i, article in enumerate(articles[:5], 1):  # 각 섹션에서 상위 5개 기사만 출력
                # 제목 추출
                title_tag = article.select_one('h2.tit')
                title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
                
                # 링크 추출
                link_tag = article.find('a')
                link = urljoin(section_url, link_tag['href']) if link_tag and 'href' in link_tag.attrs else "링크 없음"
                
                # 이미지 추출
                img_tag = article.select_one('img')
                if img_tag and 'src' in img_tag.attrs:
                    img_url = urljoin(section_url, img_tag['src'])
                    # 이미지가 없는 경우를 대비한 onerror 속성 확인
                    if 'onerror' in img_tag.attrs:
                        # onerror 속성에서 실제 이미지 URL 추출
                        onerror = img_tag['onerror']
                        if 'ImgError' in onerror:
                            # ImgError 함수에서 실제 이미지 URL 추출
                            parts = onerror.split(',')
                            if len(parts) >= 2:
                                actual_url = parts[1].strip().strip("'\"")
                                img_url = urljoin(section_url, actual_url)
                    
                    try:
                        # 이미지 출력
                        display(Image(url=img_url, width=100))
                        print(f"이미지 URL: {img_url}")
                    except:
                        print("이미지를 불러올 수 없습니다.")
                else:
                    print("이미지 없음")
                
                print(f"{i}. 제목: {title}")
                print(f"   링크: {link}")
                print("-" * 50)
                
        except Exception as e:
            print(f"{section_name} 섹션을 가져오는 중 오류 발생: {e}")

# 함수 실행
scrape_nate_news()

In [ ]:
#하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기

import os
import requests
from bs4 import BeautifulSoup

def download_one_episode(title, no, url):
    # 이미지 저장 디렉토리 생성
    dir_path = f"img/{title}/{no}"
    os.makedirs(dir_path, exist_ok=True)
    
    # 웹툰 페이지 가져오기
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 이미지 태그 찾기
    img_tags = soup.select('#comic_view_area img')
    
    # 각 이미지 다운로드
    for idx, img in enumerate(img_tags):
        img_url = img['src']
        
        # 이미지 다운로드
        img_data = requests.get(img_url, headers=headers).content
        
        # 파일 저장 (이미지 확장자는 jpg로 가정)
        file_path = f"{dir_path}/{idx+1}.jpg"
        with open(file_path, 'wb') as f:
            f.write(img_data)
        
        print(f"Downloaded: {file_path}")
    
    print(f"All images downloaded to {dir_path}")

# 예시 사용법
download_one_episode('잔불의 기사', 198, 'https://comic.naver.com/webtoon/detail?titleId=768536&no=198&week=mon')